# ENGLISH DATASET

## Create the BM25 INDEX

In [1]:
pip install rank-bm25


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
from rank_bm25 import BM25Okapi
import pickle
import spacy
import re

In [15]:
import json
from rank_bm25 import BM25Okapi
import pickle
import spacy
import re

nlp = spacy.load("en_core_web_sm")

custom_stopwords = set(["https", "http", "www", "html", "com", "st", "rd", "nd", "th"])

def custom_preprocessor(text):

    text = text.replace('\n', ' ').replace('\t', ' ')
    text = text.replace('\\n', ' ').replace('\\t', ' ')
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\d+", " ", text) 
    return text.lower()

def custom_tokenizer(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.lemma_.strip() and token.lemma_.lower() not in custom_stopwords and len(token.lemma_) > 1]

def create_bm25_index(json_file, output_file):
    document_set = set()
    document_list = []

    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for key, query in data.items():
        for doc in query["top_docs"]:
            if doc["0"] not in document_set:
                document_set.add(doc["0"])
                document_list.append(doc["0"])
        for doc in query["top_docs_clarif"]:
            if doc[0] not in document_set:
                document_set.add(doc[0])
                document_list.append(doc[0])
    print("Total unique doucment sin corpus is:", len(document_set))
    print("Document list created, strat tokenization ...")
    tokenized_corpus = [custom_tokenizer(custom_preprocessor(doc)) for doc in document_list]
    print("Docuemnt cleaned and tokenized, start index bm25 ...")
    bm25 = BM25Okapi(tokenized_corpus)

    with open(f"{output_file}.pkl", 'wb') as f:
        pickle.dump((bm25, document_list, tokenized_corpus), f)

    return bm25, document_list

In [ ]:
bm25, document_list= create_bm25_index("training_top20_qulac_en_noduplicates.json", "EN_bm25")

In [16]:
def load_bm25_index(filename):
    try:
        with open(filename, 'rb') as f:
            bm25, document_list, tokenized_corpus = pickle.load(f) #changed filename
        return bm25, document_list, tokenized_corpus #and added tokenized corpus to the return
    except FileNotFoundError:
        print(f"Error: File not found.")
        return None, None, None, False

In [23]:
bm25, document_list, tokenized_corpus =load_bm25_index("EN_bm25.pkl")

In [17]:
# fucntion to get reesults from the query
def get_bm25_results(bm25_index, query, document_list, top_n=20):
    tokenized_query = custom_tokenizer(custom_preprocessor(query))
    doc_scores = bm25_index.get_scores(tokenized_query)
    ranked_docs = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)
    top_docs = [(doc_index, doc_scores[doc_index]) for doc_index in ranked_docs[:top_n]]
    return top_docs

In [18]:
#cree les vrais jugements de pertinence. 1 pour les top20 des clarif

def create_qrels(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)

    output_filename=filename.split("_noduplicates_bm25indexed.json")[0]+"_QRELS"
    with open(output_filename, "w") as run_file:
        for key, query in data.items():
            for doc in query["top_docs_clarif"]:
                doc_id=doc[2]
                run_file.write(f"{key} 0 {doc_id} 1\n")
    print("finished printing qrels")

In [43]:
create_qrels("training_top20_qulac_en_noduplicates_bm25indexed.json")

finished printing qrels


In [27]:
# prepare id maps for les docs, les ids c'est juste l'index de bmm25 
# doc_id_map = {f"doc_ID{i}": doc for i, doc in enumerate(document_list)}


In [19]:
import json
#adjuster les donnees globales pour inclure les index des docs

def add_bm25_indices(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    document_set = set()
    document_list = []
    for key, query in data.items():
        for doc in query["top_docs"]:
            if doc["0"] not in document_set:
                document_set.add(doc["0"])
                document_list.append(doc["0"])
          
            doc["3"]=f"doc_ID{document_list.index(doc['0'])}"
        for doc in query["top_docs_clarif"]:
            if doc[0] not in document_set:
                document_set.add(doc[0])
                document_list.append(doc[0])
             
            doc.append(f"doc_ID{document_list.index(doc[0])}")

    output_filename=filename.split('.json')[0]
    with open(f"{output_filename}_bm25indexed.json", "w") as f:
        json.dump(data, f, indent=2)
    print(f"Files saved in {output_filename}_bm25indexed.json ")


In [ ]:
add_bm25_indices("training_top20_qulac_en_noduplicates.json")

In [20]:

#cerate the run fie tha cotnaisn for each query the , the ranking of the top 20 docs 
# def run_rank_calculations(filename, bm25_index, document_list):
#     with open(filename, "r", encoding="utf-8") as f:
#         data = [json.loads(line) for line in f]

    
#     output_filename = f"{filename.split('.jsonl')[0]}_bm25_run.txt"
#     run_name=filename.split(".jsonl")[0]
#     with open(output_filename, "w") as run_file:
#         for query_id, item in enumerate(data):
#             # print(f"run bm25 for query {query_id}")
#             predicted_query = item["predicted"] 
#             top_docs=get_bm25_results(bm25_index, predicted_query, document_list)
            
#             # Write to the run file in TREC format
#             for rank, (doc_idx, score) in enumerate(top_docs):
#                 doc_id = f"doc_ID{doc_idx}"
#                 run_file.write(f"{query_id} Q0 {doc_id} {rank+1} {score} {run_name} \n")

#     print(f"Document de metriques de ranking cree {output_filename}")


#focntion qui va venir creer le fichier de run qui associe a chaque query ses docs pertineents 
def run_rank_calculations(filename, bm25_index, document_list, original_qids):
    with open(filename, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]

    output_filename = f"{filename.split('.jsonl')[0]}_bm25_run.txt"
    run_name = filename.split(".jsonl")[0]
    with open(output_filename, "w") as run_file:
        for i, item in enumerate(data):
            query_id = original_qids[i]  # use original query id here
            # print(f"run bm25 for query {query_id}")
            predicted_query = item["predicted"] 
            top_docs = get_bm25_results(bm25_index, predicted_query, document_list)
            
            for rank, (doc_idx, score) in enumerate(top_docs):
                doc_id = f"doc_ID{doc_idx}"
                run_file.write(f"{query_id} Q0 {doc_id} {rank+1} {score} {run_name}\n")

    print(f"Document run filename= {output_filename}")
    return output_filename


In [51]:
run_rank_calculations("EN_LLM_predictions_FEWSHOT_TOP5DOCS_TFIDF_CLEANED_metricsready.jsonl",bm25, document_list)

run bm25 for query 0
run bm25 for query 1
run bm25 for query 2
run bm25 for query 3
run bm25 for query 4
run bm25 for query 5
run bm25 for query 6
run bm25 for query 7
run bm25 for query 8
run bm25 for query 9
run bm25 for query 10
run bm25 for query 11
run bm25 for query 12
run bm25 for query 13
run bm25 for query 14
run bm25 for query 15
run bm25 for query 16
run bm25 for query 17
run bm25 for query 18
run bm25 for query 19
run bm25 for query 20
run bm25 for query 21
run bm25 for query 22
run bm25 for query 23
run bm25 for query 24
run bm25 for query 25
run bm25 for query 26
run bm25 for query 27
run bm25 for query 28
run bm25 for query 29
run bm25 for query 30
run bm25 for query 31
run bm25 for query 32
run bm25 for query 33
run bm25 for query 34
run bm25 for query 35
run bm25 for query 36
run bm25 for query 37
run bm25 for query 38
run bm25 for query 39
run bm25 for query 40
run bm25 for query 41
run bm25 for query 42
run bm25 for query 43
run bm25 for query 44
run bm25 for query 4

In [46]:
pip install pytrec_eval


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp311-cp311-linux_x86_64.whl size=303731 sha256=586be8273e1c7f29d7d6b5ffc6f60cb88a0743fd82627ba4001c7818f48e92c4
  Stored in directory: /users/nfs/Etu0/21402600/.cache/pip/wheels/0f/89/42/86aecdb99975f1840c27bc37fdfed72116abcf82e2c9dc76a8
Successfully built pytrec_eval

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## evaluate metrics 

In [8]:
from sklearn.model_selection import KFold

#fucntion to split dataset into folds
def generate_folds(tokenized_dataset, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    folds = list(kf.split(tokenized_dataset))
    return folds


In [21]:
#fucntion that calculates the metrics between the qrels file and the predcition file
import pytrec_eval
import numpy as np

def get_metrics_dict(qrels_file, run_file):
    # Load qrels
    qrels = {}
    with open(qrels_file, "r") as f:
        for line in f:
            query_id, _, doc_id, relevance = line.strip().split()
            qrels.setdefault(query_id, {})[doc_id] = int(relevance)

    # Load run file
    run = {}
    with open(run_file, "r") as f:
        for line in f:
            query_id, _, doc_id, rank, score, _ = line.strip().split()
            run.setdefault(query_id, {})[doc_id] = float(score)

    metrics = {'map', 'ndcg', 'recall', 'recip_rank', 'P'}  # includes precision@k and MRR

    evaluator = pytrec_eval.RelevanceEvaluator(qrels, metrics)
    results = evaluator.evaluate(run)

    # Aggregate
    all_metrics = {}
    for query_id, query_scores in results.items():
        for metric, value in query_scores.items():
            all_metrics.setdefault(metric, []).append(value)

    averaged_metrics = {metric: np.mean(values) for metric, values in all_metrics.items()}
    print(averaged_metrics)
    return averaged_metrics


In [22]:
import numpy as np
def evaluate_cross_val(filenameOG, all_qrels_file , bm25_index, document_list):
    all_fold_metrics = []
    with open(filenameOG, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    folds = generate_folds(data)

    final_metric_output="RI_METRICS_"+filenameOG.split("_metricsready.jsonl")[0]
    with open(f"{final_metric_output}.txt", "w") as output_file:
        for fold_num, (train_indices, test_indices) in enumerate(folds):
            
            print(f"\n=== Fold {fold_num + 1} ===")
            print(f"Size of fold{fold_num} is of size {len(test_indices)}")
            print(f"Indices are {test_indices}")
            test_data = [data[i] for i in test_indices]
    
            original_qids = test_indices  # these are the original IDs
    
            fold_file = f"fold_{fold_num+1}_test.jsonl" # write the fol in a speerate file 
            with open(fold_file, "w") as f:
                for item in test_data:
                    f.write(json.dumps(item) + "\n")
    
            run_filename=run_rank_calculations(fold_file, bm25_index, document_list, original_qids) #get the q runs for the fold
    
            test_query_ids = set(str(i) for i in test_indices) #just convert test indices to string
    
            #split the original qrels file into the appropriate folds
            fold_qrels_file = f"fold_{fold_num+1}_qrels.txt"
            with open(all_qrels_file, "r") as full_qrels, open(fold_qrels_file, "w") as out_qrels:
                for line in full_qrels:
                    query_id, _, doc_id, relevance = line.strip().split()
                    if query_id in test_query_ids:
                        out_qrels.write(line)
        
    
            fold_metrics = get_metrics_dict(fold_qrels_file, run_filename)
            all_fold_metrics.append(fold_metrics)
            
            fold_line = f"Fold {fold_num + 1}: " + ", ".join([f"{k}: {v:.4f}" for k, v in fold_metrics.items()])
            output_file.write(fold_line + "\n")
    
        #aggregate across folds s
        aggregated = {}
        for metric in all_fold_metrics[0].keys():
            values = [fold[metric] for fold in all_fold_metrics]
            aggregated[metric] = np.mean(values)
    
        print("\n=== Final cross-fold averages ===")
        for metric, val in aggregated.items():
            print(f"{metric}: {val:.4f}")
        avg_line = "Average: " + ", ".join([f"{k}: {v:.4f}" for k, v in aggregated.items()])
        output_file.write(avg_line + "\n")

    return aggregated


In [30]:
evaluate_cross_val("crossval_T5_TINY_TOP5DOCS_metricsready.jsonl", "training_top20_qulac_en_QRELS" , bm25, document_list)


=== Fold 1 ===
Size of fold0 is of size 120
Indices are [  2   6  10  11  24  29  30  39  55  60  63  70  72  73  76  77  78  79
  81  82  83  86  90 101 104 109 110 118 132 135 137 140 148 155 163 165
 176 181 182 184 192 195 199 208 209 210 212 218 228 234 235 238 245 248
 257 259 268 275 280 281 284 286 300 304 312 316 329 335 340 349 356 361
 364 368 369 370 371 374 399 412 414 419 423 426 429 432 434 437 439 441
 453 464 469 473 490 494 506 515 534 535 536 537 538 545 547 554 555 558
 564 569 571 574 583 586 587 588 590 594 595 597]
Document run filename= fold_1_test_bm25_run.txt
{'map': 0.07184877406915309, 'recip_rank': 0.16100154389241078, 'P_5': 0.13, 'P_10': 0.12583333333333335, 'P_15': 0.11944444444444444, 'P_20': 0.115, 'P_30': 0.07666666666666666, 'P_100': 0.023000000000000003, 'P_200': 0.011500000000000002, 'P_500': 0.004600000000000001, 'P_1000': 0.0023000000000000004, 'recall_5': 0.033779526430455224, 'recall_10': 0.06533998394679509, 'recall_15': 0.09277390780873752, 

{'map': 0.07806120065635191,
 'recip_rank': 0.18493505178640432,
 'P_5': 0.13960224089635856,
 'P_10': 0.13291036414565824,
 'P_15': 0.1273548085901027,
 'P_20': 0.12457142857142858,
 'P_30': 0.08304761904761904,
 'P_100': 0.024914285714285715,
 'P_200': 0.012457142857142858,
 'P_500': 0.004982857142857143,
 'P_1000': 0.0024914285714285715,
 'recall_5': 0.03608273157904372,
 'recall_10': 0.06916821640413655,
 'recall_15': 0.09965363773594285,
 'recall_20': 0.1297532598908373,
 'recall_30': 0.1297532598908373,
 'recall_100': 0.1297532598908373,
 'recall_200': 0.1297532598908373,
 'recall_500': 0.1297532598908373,
 'recall_1000': 0.1297532598908373,
 'ndcg': 0.12982521634487335}

# Datset FR

In [31]:
import json
from rank_bm25 import BM25Okapi
import pickle
import spacy
import unicodedata
import re

nlp = spacy.load("fr_core_news_sm")

custom_stopwords= list(nlp.Defaults.stop_words)+["https","http","www","html", "com"]

def custom_preprocessor(text):
    # Remove unwanted characters (you can tweak the regex)
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    text = text.replace('\n', ' ').replace('\t',' ')
    text = text.replace('\\n', ' ').replace('\\t',' ')
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\d+", " ", text)  # remove digits
    return text.lower()

def custom_tokenizer(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.lemma_.strip() and token.lemma_.lower() not in custom_stopwords and len(token.lemma_) > 1]

def create_bm25_index(json_file, output_file):
    document_set = set()
    document_list = []

    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for key, query in data.items():
        for doc in query["top_docs"]:
            if doc["0"] not in document_set:
                document_set.add(doc["0"])
                document_list.append(doc["0"])
        for doc in query["top_docs_clarif"]:
            if doc[0] not in document_set:
                document_set.add(doc[0])
                document_list.append(doc[0])
    print("Total unique doucment sin corpus is:", len(document_set))
    print("Document list created, strat tokenization ...")
    tokenized_corpus = [custom_tokenizer(custom_preprocessor(doc)) for doc in document_list]
    print("Docuemnt cleaned and tokenized, start index bm25 ...")
    bm25 = BM25Okapi(tokenized_corpus)

    with open(f"{output_file}.pkl", 'wb') as f:
        pickle.dump((bm25, document_list, tokenized_corpus), f)

    return bm25, document_list

In [23]:
bm25_fr, document_list_fr= create_bm25_index("FR_training_top20_qulac_noduplicates.json", "FR_bm25")

Total unique doucment sin corpus is: 10067
Document list created, strat tokenization ...
Docuemnt cleaned and tokenized, start index bm25 ...


In [27]:
add_bm25_indices("FR_training_top20_qulac_noduplicates.json") #fix les index des docs pour les avoir dans le doc original

In [32]:
#cree les vrais jugements de pertinence. 1 pour les top20 des clarif

def create_qrels_fr(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)

    output_filename=filename.split("_noduplicates_bm25indexed.json")[0]+"_QRELS"
    with open(output_filename, "w") as run_file:
        for key, query in data.items():
            for doc in query["top_docs_clarif"]:
                doc_id=doc[1]
                run_file.write(f"{key} 0 {doc_id} 1\n")
    print("finished printing qrels")

In [45]:
create_qrels_fr("FR_training_top20_qulac_noduplicates_bm25indexed.json")

finished printing qrels


In [33]:
fr_bm25, fr_document_list, fr_tokenized_corpus =load_bm25_index("FR_bm25.pkl")

In [36]:
evaluate_cross_val("FR_crossval_T5_SMALL_TOP5DOCS_TFIDF_metricsready.jsonl", "FR_training_top20_qulac_QRELS" , fr_bm25, fr_document_list)


=== Fold 1 ===
Size of fold0 is of size 120
Indices are [  2   6   9  10  11  29  30  39  55  60  63  70  72  73  76  77  78  79
  81  82  83  84  86  90 101 104 110 118 131 132 135 140 145 148 153 158
 163 165 177 181 182 184 188 196 199 204 208 209 210 211 218 231 234 235
 238 247 248 250 257 265 271 274 277 278 281 284 286 289 291 299 300 302
 304 309 312 331 340 352 354 355 357 361 364 367 368 388 394 395 419 431
 433 437 442 444 449 451 462 464 465 468 490 501 512 524 533 534 536 542
 552 555 556 557 566 571 580 582 584 587 591 592]
Document run filename= fold_1_test_bm25_run.txt
{'map': 0.0019889168386652907, 'recip_rank': 0.0175, 'P_5': 0.006666666666666667, 'P_10': 0.005, 'P_15': 0.0038888888888888883, 'P_20': 0.004166666666666667, 'P_30': 0.002777777777777778, 'P_100': 0.0008333333333333334, 'P_200': 0.0004166666666666667, 'P_500': 0.00016666666666666666, 'P_1000': 8.333333333333333e-05, 'recall_5': 0.0016666666666666668, 'recall_10': 0.0025, 'recall_15': 0.002916666666666666

{'map': 0.00336551051811727,
 'recip_rank': 0.014950640862405568,
 'P_5': 0.007719887955182074,
 'P_10': 0.007218487394957984,
 'P_15': 0.006940242763772175,
 'P_20': 0.006379551820728291,
 'P_30': 0.004253034547152194,
 'P_100': 0.0012759103641456584,
 'P_200': 0.0006379551820728292,
 'P_500': 0.0002551820728291316,
 'P_1000': 0.0001275910364145658,
 'recall_5': 0.0019800973020787263,
 'recall_10': 0.003705071502285125,
 'recall_15': 0.005310346945795862,
 'recall_20': 0.006497985158975872,
 'recall_30': 0.006497985158975872,
 'recall_100': 0.006497985158975872,
 'recall_200': 0.006497985158975872,
 'recall_500': 0.006497985158975872,
 'recall_1000': 0.006497985158975872,
 'ndcg': 0.007111986990638726}